In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [2]:
names_df = pd.read_excel('./gu_list/서울_강남구.xlsx')
names_df

,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment
1,강남구,논현동,숯칼 도산 숯닭갈비,한식,202545.473911,446358.225573,https://place.map.kakao.com/713234231#comment
2,강남구,도곡동,롯데시네마 도곡,카페,204098.596031,442807.317225,https://place.map.kakao.com/1913617889#comment
3,강남구,신사동,이소젠 네타,카페,201713.280105,446129.893616,https://place.map.kakao.com/1688463314#comment
4,강남구,신사동,로지,일식,203261.244008,446841.847957,https://place.map.kakao.com/11598105#comment
...,...,...,...,...,...,...,...
8949,강남구,삼성동,크랩52,양식,205340.631122,445354.571117,https://place.map.kakao.com/1331156078#comment
8950,강남구,신사동,원조마포소금구이 강남점,한식,201782.101227,446748.067184,https://place.map.kakao.com/888714552#comment
8951,강남구,삼성동,어게인,카페,204931.260579,445317.144016,https://place.map.kakao.com/1788711084#comment
8952,강남구,신사동,릿라운지,카페,203311.068042,447204.468862,https://place.map.kakao.com/839892336#comment


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

source_url = "https://map.kakao.com/"

page_urls = []

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in tqdm_notebook(names_df['0']):
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source

        soup = BeautifulSoup(html, "html.parser")
        moreview = soup.find(name="a", attrs={"class":"moreview"})
    
        page_url = moreview.get("href")
        print(page_url)
        page_urls.append(page_url)
    except:
        pass

driver.close()

https://place.map.kakao.com/207990934
https://place.map.kakao.com/1441083691
https://place.map.kakao.com/1605389572
https://place.map.kakao.com/908713570
https://place.map.kakao.com/16738793
https://place.map.kakao.com/21413534
https://place.map.kakao.com/1824464463
https://place.map.kakao.com/15558781
https://place.map.kakao.com/21753236
https://place.map.kakao.com/8607176
https://place.map.kakao.com/274655501
https://place.map.kakao.com/7902292
https://place.map.kakao.com/16296316


In [3]:
page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

In [4]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            if review != ' ':
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6):
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                if review != ' ':
                    row = [names.text, rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=review_df.columns)
                    review_df = review_df.append(series, ignore_index=True)
    except:
        pass
    
driver.close()

In [ ]:
review_df['review'].replace(' ', np.nan, inplace=True)
review_data = review_df.dropna(subset=['review'])
review_data

In [ ]:
review_data.to_csv("review_data.csv", index=False)

: 

In [ ]:
review_data.to_csv("./gu_list/강남구_review_data.csv", index=False)